<a href="https://colab.research.google.com/github/Bibhash123/JSTARS-NESNet/blob/main/Validation_%26_Benchmarking/SO2_and_O3_Benchmarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files
_ = files.upload()
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d bibhash123/so2estimation
!unzip so2estimation.zip -d "/content/SO2dataset"
!rm so2estimation.zip

!kaggle datasets download -d bibhash123/o3estimation
!unzip o3estimation.zip -d "/content/O3dataset"
!rm o3estimation.zip
from IPython.display import clear_output
clear_output(wait=False)

In [ ]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from sklearn.metrics import mean_squared_error,mean_absolute_error

SEED=123
def seedAll(seed):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ["PYTHONHASHSEED"]=str(seed)

seedAll(SEED)

In [ ]:
def pearson(y_true,y_pred):
  if len(y_true.shape)!=1:
    true = []
    for i in range(y_true.shape[0]):
      true.extend(y_true[i])
    pred = []
    for i in range(y_pred.shape[0]):
      pred.extend(y_pred[i])
  else:
    true=y_true
    pred=y_pred
  return np.mean((np.array(true)-np.mean(true))*(np.array(pred)-np.mean(pred)))/(np.std(np.array(true))*np.std(np.array(pred)))

def pearsonCorrAvgDays(true,pred):
  # assert len(true.shape)>=3,"true must have at least 3 dimensions, found {}".format(len(true.shape))
  assert true.shape==pred.shape, "true and pred must have same shape, found {} and {}".format(true.shape,pred.shape)
  scores = []
  for i in range(true.shape[0]):
    scores.append(pearson(true[i],pred[i]))
  return np.mean(scores),scores

def pearsonCorrAvgPixels(true,pred):
  scores = []
  for i in range(true.shape[1]):
    scores.append(pearson(true[:,i],pred[:,i]))
  return np.mean(scores),scores

In [ ]:
def loadData(df,satdir = "/content/SO2dataset/satellite/",gdir = "/content/SO2dataset/ground/"):
  X = []
  Y = []
  for i in range(df.shape[0]):
    factor = 46*(6.02214/6.023)*1e2
    sat = np.expand_dims(factor*np.load(os.path.join(satdir,df["SatFile"].iloc[i])),axis=2).flatten()      
    ground = np.load(os.path.join(gdir,df["GroundFile"].iloc[i])).flatten()
    if not np.isnan(np.sum(sat)) and not np.isnan(np.sum(ground)):
      if not np.std(ground)==0:
        X.append(sat)
        Y.append(ground)
  return np.stack(X,axis=0).flatten(),np.stack(Y,axis=0).flatten()


def getDayWise(Y,out_shape=3283):
  x = Y.shape[0]//out_shape
  day_wise = []
  for i in range(1,x+1,1):
    day_wise.append(Y[(i-1)*out_shape:i*out_shape])
  return np.stack(day_wise,axis=0)

# SO<sub>2</sub>

In [ ]:
files = pd.read_csv("/content/SO2dataset/files.csv").sample(frac=1).reset_index(drop=True)

In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files)
  X_val,Y_val = loadData(val_files)
  # loss_plt = utils.loss_plt()
  model = LinearRegression()
  model.fit(X_train.reshape(-1,1),Y_train.reshape(-1,1))

  pred = model.predict(X_val.reshape(-1,1))
  rmse = mean_squared_error(Y_val,pred,squared=False)
  mae = mean_absolute_error(Y_val,pred)
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(pred)[:,:,0])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 7.9204208542601275
Fold 0 Pearson coeff avg over days: 0.0069865748637057135

Fold 1

Fold 1 RMSE Score: 8.839438013724363
Fold 1 Pearson coeff avg over days: -0.004134171806520218

Fold 2

Fold 2 RMSE Score: 8.442438430857843
Fold 2 Pearson coeff avg over days: -0.006444729770980234

Fold 3

Fold 3 RMSE Score: 8.409505949049278
Fold 3 Pearson coeff avg over days: 0.0056445435171737285

Fold 4

Fold 4 RMSE Score: 6.34009426757259
Fold 4 Pearson coeff avg over days: 0.0030477021168983692

CV RMSE Score: 7.990379503092839

CV MAE Score: 4.791578595657915

CV Pearson coeff avg over days: 0.001019983784055472


In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files)
  X_val,Y_val = loadData(val_files)
  # loss_plt = utils.loss_plt()
  model = XGBRegressor(objective="reg:squarederror")
  model.fit(X_train.reshape(-1,1),Y_train.reshape(-1,1))

  rmse = mean_squared_error(Y_val,model.predict(X_val.reshape(-1,1)),squared=False)
  mae = mean_absolute_error(Y_val,model.predict(X_val.reshape(-1,1)))
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(model.predict(X_val.reshape(-1,1)))[:,:])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 7.9239971300631735
Fold 0 Pearson coeff avg over days: -0.029171490393040414

Fold 1

Fold 1 RMSE Score: 8.82099461611264
Fold 1 Pearson coeff avg over days: 0.002642456767600125

Fold 2

Fold 2 RMSE Score: 8.438140515872915
Fold 2 Pearson coeff avg over days: -0.018783030073169952

Fold 3

Fold 3 RMSE Score: 8.393705743975607
Fold 3 Pearson coeff avg over days: -0.01678525179868811

Fold 4

Fold 4 RMSE Score: 6.326457468715553
Fold 4 Pearson coeff avg over days: -0.012840253712497949

CV RMSE Score: 7.9806590949479785

CV MAE Score: 4.79628035680234

CV Pearson coeff avg over days: -0.01498751384195926


In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files)
  X_val,Y_val = loadData(val_files)
  # loss_plt = utils.loss_plt()
  model = LGBMRegressor()
  model.fit(X_train.reshape(-1,1),Y_train.flatten())

  rmse = mean_squared_error(Y_val,model.predict(X_val.reshape(-1,1)),squared=False)
  mae = mean_absolute_error(Y_val,model.predict(X_val.reshape(-1,1)))
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(model.predict(X_val.reshape(-1,1)))[:,:])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 7.92479158051026
Fold 0 Pearson coeff avg over days: -0.027624426525498767

Fold 1

Fold 1 RMSE Score: 8.822023468789544
Fold 1 Pearson coeff avg over days: -0.00028239848633404384

Fold 2

Fold 2 RMSE Score: 8.439987520140201
Fold 2 Pearson coeff avg over days: -0.01787699025003806

Fold 3

Fold 3 RMSE Score: 8.393209290321867
Fold 3 Pearson coeff avg over days: -0.014910459606768336

Fold 4

Fold 4 RMSE Score: 6.326256916906433
Fold 4 Pearson coeff avg over days: -0.01155473440569013

CV RMSE Score: 7.981253755333659

CV MAE Score: 4.796563385518146

CV Pearson coeff avg over days: -0.014449801854865866


# O<sub>3</sub>

In [ ]:
files = pd.read_csv("/content/O3dataset/files.csv").sample(frac=1).reset_index(drop=True)

In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  X_val,Y_val = loadData(val_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  # loss_plt = utils.loss_plt()
  model = LinearRegression()
  model.fit(X_train.reshape(-1,1),Y_train.reshape(-1,1))

  pred = model.predict(X_val.reshape(-1,1))
  rmse = mean_squared_error(Y_val,pred,squared=False)
  mae = mean_absolute_error(Y_val,pred)
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(pred)[:,:,0])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 20.118218210171975
Fold 0 Pearson coeff avg over days: 0.11673511181248238

Fold 1

Fold 1 RMSE Score: 19.43209885819915
Fold 1 Pearson coeff avg over days: 0.1607236160481196

Fold 2

Fold 2 RMSE Score: 18.78530527708602
Fold 2 Pearson coeff avg over days: 0.06447347603914587

Fold 3

Fold 3 RMSE Score: 20.274511083835538
Fold 3 Pearson coeff avg over days: 0.096551399478444

Fold 4

Fold 4 RMSE Score: 18.187697128564025
Fold 4 Pearson coeff avg over days: 0.09618200286161274

CV RMSE Score: 19.359566111571343

CV MAE Score: 14.039631955296338

CV Pearson coeff avg over days: 0.10693312124796092


In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  X_val,Y_val = loadData(val_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  
  # loss_plt = utils.loss_plt()
  model = XGBRegressor(objective="reg:squarederror")
  model.fit(X_train.reshape(-1,1),Y_train.reshape(-1,1))

  rmse = mean_squared_error(Y_val,model.predict(X_val.reshape(-1,1)),squared=False)
  mae = mean_absolute_error(Y_val,model.predict(X_val.reshape(-1,1)))
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(model.predict(X_val.reshape(-1,1)))[:,:])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 20.070831922876
Fold 0 Pearson coeff avg over days: 0.07738052422231878

Fold 1

Fold 1 RMSE Score: 19.642754979826393
Fold 1 Pearson coeff avg over days: 0.13167360451008012

Fold 2

Fold 2 RMSE Score: 18.281353967293644
Fold 2 Pearson coeff avg over days: 0.06771333649876042

Fold 3

Fold 3 RMSE Score: 20.665961606353843
Fold 3 Pearson coeff avg over days: 0.07797344097309206

Fold 4

Fold 4 RMSE Score: 18.778353640296498
Fold 4 Pearson coeff avg over days: 0.04444347879845657

CV RMSE Score: 19.487851223329276

CV MAE Score: 14.156983315948713

CV Pearson coeff avg over days: 0.07983687700054158


In [ ]:
scores_list = []
rmses = []
maes = []
pearsons = []

for fold in range(5):
  print("\nFold {}\n".format(fold))
  train_files = files[files["Fold"]!=fold]
  val_files = files[files["Fold"]==fold]

  X_train,Y_train = loadData(train_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  X_val,Y_val = loadData(val_files,satdir="/content/O3dataset/satellite/",gdir='/content/O3dataset/ground/')
  
  # loss_plt = utils.loss_plt()
  model = LGBMRegressor()
  model.fit(X_train.reshape(-1,1),Y_train.flatten())

  rmse = mean_squared_error(Y_val,model.predict(X_val.reshape(-1,1)),squared=False)
  mae = mean_absolute_error(Y_val,model.predict(X_val.reshape(-1,1)))
  rmses.append(rmse)
  maes.append(mae)

  print("Fold {} RMSE Score: {}".format(fold, rmse))
  s,ls = pearsonCorrAvgDays(getDayWise(Y_val),getDayWise(model.predict(X_val.reshape(-1,1)))[:,:])
  r2 = np.mean([i for i in ls if not pd.isnull(i)])
  pearsons.append(r2)

  print("Fold {} Pearson coeff avg over days: {}".format(fold,r2))
  scores_list.append(ls)
print("\nCV RMSE Score: {}".format(np.mean(rmses)))
print("\nCV MAE Score: {}".format(np.mean(maes)))
print("\nCV Pearson coeff avg over days: {}".format(np.mean(pearsons)))


Fold 0

Fold 0 RMSE Score: 20.134046704701024
Fold 0 Pearson coeff avg over days: 0.06672528943658734

Fold 1

Fold 1 RMSE Score: 19.66018900461929
Fold 1 Pearson coeff avg over days: 0.11274924010662216

Fold 2

Fold 2 RMSE Score: 18.373409477590926
Fold 2 Pearson coeff avg over days: 0.06506208937333358

Fold 3

Fold 3 RMSE Score: 20.691541707484227
Fold 3 Pearson coeff avg over days: 0.07003300605251478

Fold 4

Fold 4 RMSE Score: 18.79530848108473
Fold 4 Pearson coeff avg over days: 0.040891065048074764

CV RMSE Score: 19.530899075096038

CV MAE Score: 14.194040271373169

CV Pearson coeff avg over days: 0.07109213800342652
